<a href="https://colab.research.google.com/github/syauqyideas/DualSPHysic_Colab_Notebook_Runner/blob/main/DualSPHPhysic_Fixed_Public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

name = 'SuperResult' #@param {type: 'string'}
os.environ['name'] = name
DualsphysicsVersion = '5.2.2' #@param ['4','5.2.2']{allow-input : false}

if DualsphysicsVersion == '5.2.2':
  os.environ['pathsph'] = '/content/DualSPHysics_Linux-main/EXECS5'
  os.environ["LD_LIBRARY_PATH"] = "/content/DualSPHysics_Linux-main/src/lib/linux_gcc"# + os.environ.get("LD_LIBRARY_PATH", "")
else:
  os.environ['pathsph'] = '/content/DualSPHysics_Linux-main/EXECS'

runner_directory ='/content/DualSPHysics_Linux-main/RUN_DIRECTORY/'

github_project = True #@param {type: 'boolean'}
github_project_PAT = True #@param {type: 'boolean'}
personal_project_repo = "syauqy_dualsphproject" #@param {type: 'string'}
github_username = "syauqyideas" #@param {type: 'string'}
if github_project_PAT == True :
  github_personal_token = "" #@param {type: 'string'}
your_case_path = f'/content/{personal_project_repo}/{name}'

run_directory = runner_directory + name
os.environ['running_def'] = f'{name}_Def'
#if you use personal .sh file :
personal_sh_file = False #@param {type : 'boolean'}
if personal_sh_file == True :
  FileRunnerName = "" #@param {type: 'string'}

<h3>Google Drive Connect

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h3> Setting Environment

In [ ]:
!apt install nvidia-driver-555 #i dont know, colab and dualsphysic is just didnt recognize each other - so we need to install cuda that compatible -- the 555 version is best though
!sudo apt update && sudo apt upgrade
!pip install IPython
import IPython #at some point, i just bored to see the dirty output
from IPython.display import clear_output
clear_output() #you can take away this

!nvcc --version #it is just check cuda version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


<h3>Github Download Dualsphysic and clone your own project

In [ ]:

!wget https://github.com/syauqyideas/DualSPHysics_Linux/archive/refs/heads/main.zip
!unzip main.zip
!rm main.zip
clear_output()

if github_project == True :
  if github_project_PAT == True :
  !git clone https://{github_personal_token}@github.com/{github_username}/{personal_project_repo}.git
  else :
  !git clone https://github.com/{github_username}/{public_project_repo}.git
  !unzip main.zip
  clear_output()

!sudo chmod +x /content/DualSPHysics_Linux-main/EXECS5/*
!sudo chmod +x /content/DualSPHysics_Linux-main/EXECS5/FlexStruc/*
!sudo chmod +x /content/DualSPHysics_Linux-main/EXECS5/DSNNewtonian/*
!sudo chmod +x /content/DualSPHysics_Linux-main/EXECS5/DSGcc7/*

!sudo chmod +x /content/DualSPHysics_Linux-main/EXECS/GenCase4_linux64
!sudo chmod +x /content/DualSPHysics_Linux-main/EXECS/DualSPHysics4CPU_linux64
!sudo chmod +x /content/DualSPHysics_Linux-main/EXECS/BoundaryVTK4_linux64
!sudo chmod +x /content/DualSPHysics_Linux-main/EXECS/PartVTK4_linux64
!sudo chmod +x /content/DualSPHysics_Linux-main/EXECS/PartVTKOut4_linux64
!sudo chmod +x /content/DualSPHysics_Linux-main/EXECS/MeasureTool4_linux64
!sudo chmod +x /content/DualSPHysics_Linux-main/EXECS/ComputeForces4_linux64
!sudo chmod +x /content/DualSPHysics_Linux-main/EXECS/DualSPHysics4_linux64


<h4> Setting Runner Environment, copy file to RUN_DIRECTORY

In [ ]:
!rm -r {runner_directory}/{name}
!cp -r {your_case_path} {runner_directory}

!cd /content

os.environ['dirout'] = f'{run_directory}/{name}_out'
os.environ['diroutdata'] = f'{run_directory}/data'
os.environ['dirout2'] = f'{run_directory}/out'
os.environ['surface'] = f'{run_directory}/Surface'
os.environ['particle'] = f'{run_directory}/particle'

if DualsphysicsVersion == '5.2.2':
  os.environ['gencase'] = "GenCase_linux64"
  os.environ["DualSPHysics"] = "DualSPHysics5.2_linux64"
  os.environ["vtk"] = "PartVTK_linux64"
  os.environ["isosurface"] = "IsoSurface_linux64"

elif DualsphysicsVersion == '4':
  os.environ['gencase'] = "GenCase_linux64"
  os.environ["DualSPHysics"] = "DualSPHysics4_linux64"
  os.environ["vtk"] = "PartVTK4_Linux64"
  os.environ["isosurface"] = "IsoSurface4_linux64"

<h3><strong> Run using the internal bash command line</strong></h3>
<h5>comment the process you didnt want to run

In [ ]:
%cd $run_directory

#preprocessing case with gencase
!mkdir $dirout
!$pathsph/$gencase $running_def $dirout -save:all

#running the case
!mkdir $diroutdata
!$pathsph/$DualSPHysics -gpu $dirout -dirdataout data -svres

#running VTK creator
%cd $run_directory
!mkdir $particle
#%cd $particle
!$pathsph/$vtk -dirin $diroutdata -savevtk  $particle/particle -onlytype:-all,+fluid


#running isosurface
%cd $run_directory
!mkdir $surface
!$pathsph/$isosurface -dirin $diroutdata -saveiso $surface/Surface

<h3> Run with external command line -- configured with your runner file  (.sh file) </h3>

In [ ]:
%cd $CaseFilePath
!sudo bash ./$FileRunnerName

Export

<h4> In case you need to edit xml, just use nano

In [ ]:
!pip install colab-xterm
%load_ext colabxterm
!apt install nano
%cd $run_directory
clear_output()
%xterm